In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
df = pd.read_csv('/home/jupyter-a.kosheleva-14/KC case product analitycs.csv', parse_dates=['date'])

In [4]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [5]:
df[(df.date >= '2020-01-01') & (df.date < '2020-02-01')].query('event == "app_install"').shape

(80297, 8)

In [6]:
#отобрал тух пользователей, кто совершил покупку
customers_list = df[df.event == 'purchase'].device_id.unique()

In [7]:
#отобрал датафрейм только с ними
df_customers = df.query('device_id in @customers_list') #только покупатели

In [8]:
df_customers

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-
6,2020-01-01,app_start,NaN,android,8383386,female,Saint-Petersburg,vk_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [9]:
#отобрал теперь для этих покупателей дату установки приложения
test = df_customers[df_customers.event == 'app_install'][['date', 'event', 'device_id']]

In [10]:
test

,date,event,device_id
2515061,2020-01-01,app_install,4921563
2515063,2020-01-01,app_install,23006325
2515066,2020-01-01,app_install,8423608
2515067,2020-01-01,app_install,532710
2515070,2020-01-01,app_install,528071
...,...,...,...
2669634,2020-03-31,app_install,228794
2669645,2020-03-31,app_install,20224705
2669647,2020-03-31,app_install,1434534
2669650,2020-03-31,app_install,271586


In [11]:
#добавил 7 дней к дате установки (все еще только покупатели)
test['day_7'] = test.date + dt.timedelta(days=7)

In [12]:
test

,date,event,device_id,day_7
2515061,2020-01-01,app_install,4921563,2020-01-08
2515063,2020-01-01,app_install,23006325,2020-01-08
2515066,2020-01-01,app_install,8423608,2020-01-08
2515067,2020-01-01,app_install,532710,2020-01-08
2515070,2020-01-01,app_install,528071,2020-01-08
...,...,...,...,...
2669634,2020-03-31,app_install,228794,2020-04-07
2669645,2020-03-31,app_install,20224705,2020-04-07
2669647,2020-03-31,app_install,1434534,2020-04-07
2669650,2020-03-31,app_install,271586,2020-04-07


In [13]:
#отобрал только эту дату +7 дней и айди тех, кто совершил покупку - обязательно
df_device_id_plus_dateOfExpiry = test[['device_id', 'day_7']]

In [14]:
df_device_id_plus_dateOfExpiry

,device_id,day_7
2515061,4921563,2020-01-08
2515063,23006325,2020-01-08
2515066,8423608,2020-01-08
2515067,532710,2020-01-08
2515070,528071,2020-01-08
...,...,...
2669634,228794,2020-04-07
2669645,20224705,2020-04-07
2669647,1434534,2020-04-07
2669650,271586,2020-04-07


In [15]:
#объединил с теми, кто совершил покупку таблицу с 7+ днями, чтобы эти 7+ дней появились
df_customers_7days = df_customers.merge(df_device_id_plus_dateOfExpiry, on='device_id')[['date', 'event', 'device_id', 'day_7']]

In [16]:
df_customers_7days

,date,event,device_id,day_7
0,2020-01-01,app_start,669460,2020-01-08
1,2020-01-01,search,669460,2020-01-08
2,2020-01-03,app_start,669460,2020-01-08
3,2020-01-03,choose_item,669460,2020-01-08
4,2020-01-03,search,669460,2020-01-08
...,...,...,...,...
1737560,2020-03-31,purchase,908531,2020-04-07
1737561,2020-03-31,search,908531,2020-04-07
1737562,2020-03-31,tap_basket,908531,2020-04-07
1737563,2020-03-31,app_install,908531,2020-04-07


In [17]:
#оттуда отобрал покупателей, которые для своей даты любого действия совершили его ранее, чем за 7 дней до покупки
df_customers_7days['in_time'] = df_customers_7days.date <= df_customers_7days.day_7

In [18]:
df_customers_7days

,date,event,device_id,day_7,in_time
0,2020-01-01,app_start,669460,2020-01-08,True
1,2020-01-01,search,669460,2020-01-08,True
2,2020-01-03,app_start,669460,2020-01-08,True
3,2020-01-03,choose_item,669460,2020-01-08,True
4,2020-01-03,search,669460,2020-01-08,True
...,...,...,...,...,...
1737560,2020-03-31,purchase,908531,2020-04-07,True
1737561,2020-03-31,search,908531,2020-04-07,True
1737562,2020-03-31,tap_basket,908531,2020-04-07,True
1737563,2020-03-31,app_install,908531,2020-04-07,True


In [19]:
#теперь отобрал из них тех, кто совершил эту покупку в срок - и их idшники
in_time_customers = df_customers_7days[df_customers_7days.event == 'purchase'].query('in_time == True').device_id.unique()

In [20]:
in_time_customers

array([  669460,   833621, 10144701, ..., 14204302, 18219505,   908531])

In [21]:
#теперь из исходного датафрейма отобрал тех, кто совершил эту покупку в срок
df_in_time = df.query('device_id in @in_time_customers')

In [22]:
#далее отобрал из исходного датафрейма тех, кто установил приложение, сгруппировал по дате и посчитал количество уникальных id
c = df[df.event == 'app_install'].groupby('date', as_index=False).agg({'device_id': 'nunique'})

In [23]:
c

,date,device_id
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671
...,...,...
86,2020-03-27,1199
87,2020-03-28,1091
88,2020-03-29,1117
89,2020-03-30,994


In [24]:
#теперь из тех, кто совершил покупку в срок, отобрал установку приложения, сгруппировал по дате и посчитал там уникальные id
d = df_in_time[df_in_time.event == 'app_install'].groupby('date', as_index=False).agg({'device_id': 'nunique'})

In [25]:
d

,date,device_id
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587
...,...,...
86,2020-03-27,277
87,2020-03-28,209
88,2020-03-29,223
89,2020-03-30,204


In [26]:
merged_new = c.merge(d, on='date')

In [27]:
merged_new

,date,device_id_x,device_id_y
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587
...,...,...,...
86,2020-03-27,1199,277
87,2020-03-28,1091,209
88,2020-03-29,1117,223
89,2020-03-30,994,204


In [28]:
merged_new['conv'] = merged_new.device_id_y / merged_new.device_id_x*100

In [29]:
merged_new.sort_values('conv', ascending=False)

,date,device_id_x,device_id_y,conv
0,2020-01-01,3579,1408,39.340598
8,2020-01-09,1424,558,39.185393
14,2020-01-15,4310,1650,38.283063
13,2020-01-14,5173,1973,38.140344
1,2020-01-02,3144,1186,37.722646
...,...,...,...,...
77,2020-03-18,1171,235,20.068318
88,2020-03-29,1117,223,19.964190
81,2020-03-22,1261,251,19.904837
87,2020-03-28,1091,209,19.156737


In [30]:
k = df[df.event == 'app_install'].groupby('utm_source', as_index=False).agg({'device_id': 'nunique'})

In [31]:
k

,utm_source,device_id
0,-,32460
1,facebook_ads,13916
2,google_ads,26286
3,instagram_ads,20096
4,referal,9282
5,vk_ads,23189
6,yandex-direct,29368


In [32]:
m = df_in_time[df_in_time.event == 'app_install'].groupby('utm_source', as_index=False).agg({'device_id': 'nunique'})

In [33]:
m

,utm_source,device_id
0,-,9795
1,facebook_ads,4226
2,google_ads,7964
3,instagram_ads,5925
4,referal,2802
5,vk_ads,7169
6,yandex-direct,8793


In [34]:
merged_recource = k.merge(m, on='utm_source')

In [35]:
merged_recource

,utm_source,device_id_x,device_id_y
0,-,32460,9795
1,facebook_ads,13916,4226
2,google_ads,26286,7964
3,instagram_ads,20096,5925
4,referal,9282,2802
5,vk_ads,23189,7169
6,yandex-direct,29368,8793


In [36]:
merged_recource['conv'] = merged_recource.device_id_y / merged_recource.device_id_x*100

In [37]:
merged_recource

,utm_source,device_id_x,device_id_y,conv
0,-,32460,9795,30.175601
1,facebook_ads,13916,4226,30.367922
2,google_ads,26286,7964,30.297497
3,instagram_ads,20096,5925,29.483479
4,referal,9282,2802,30.187460
5,vk_ads,23189,7169,30.915520
6,yandex-direct,29368,8793,29.940752


In [42]:
df.groupby('utm_source') \
    .agg({'purchase_sum':'median'}) \
    .sort_values(by=['purchase_sum'])

,purchase_sum
utm_source,
referal,389.0
google_ads,390.0
vk_ads,392.5
yandex-direct,393.5
facebook_ads,394.0
instagram_ads,397.0
-,397.5


In [46]:
only_transactions = df_customers[df_customers.event == 'purchase']

In [47]:
only_transactions

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-
7692,2020-01-01,purchase,374.0,android,77448,female,Saint-Petersburg,yandex-direct
7693,2020-01-01,purchase,98.5,ios,7458607,female,Moscow,instagram_ads
7694,2020-01-01,purchase,304.5,ios,3520947,male,Moscow,instagram_ads
7695,2020-01-01,purchase,153.5,android,1170605,male,Moscow,google_ads
...,...,...,...,...,...,...,...,...
2507469,2020-03-31,purchase,6546.5,android,11864116,male,Saint-Petersburg,google_ads
2507470,2020-03-31,purchase,3495.0,android,3403969,female,Saint-Petersburg,google_ads
2507471,2020-03-31,purchase,2857.5,ios,26120362,female,Moscow,google_ads
2507472,2020-03-31,purchase,5671.5,ios,22011351,female,Moscow,yandex-direct


In [49]:
only_transactions.groupby('utm_source') \
    .agg({'purchase_sum':'sum'}) \
    .sort_values(by=['purchase_sum'])

,purchase_sum
utm_source,
referal,8837044.5
facebook_ads,12249901.0
google_ads,12868276.0
yandex-direct,13915368.0
instagram_ads,14546969.0
vk_ads,16389652.5
-,21449749.5


In [51]:
(16389652.5-9553531)/9553531

0.7155596710786828

In [52]:
(14546969.0-8561626)/14546969.0

0.41144949164324196

In [54]:
(13915368.0-10491707)/13915368.0

0.2460345281562083

In [55]:
(12868276.0-10534878)/12868276.0

0.18132949588585137

In [56]:
(12249901.0-8590498)/12249901.0

0.29872918972977824

In [ ]:
(12249901.0-8590498)/12249901.0